In [ ]:
from py2neo import Graph
import pandas as pd

In [ ]:
# connect to the graph -> it should be running already
graph = Graph(scheme="bolt", host="localhost", port=7687, auth=('neo4j', '12345'))

# connect to test database, only accessible inside ipf network
# pw = ''
# graph = Graph("bolt://neo4j.loc.ipfdd.de/newsocket",auth=('neo4j',pw))

In [ ]:
# manage output: create a file logger and a stream logger with different levels
import logging
logger = logging.getLogger('neo4Polymer')
logger.setLevel("DEBUG")
# create file handler which logs even debug messages
fh = logging.FileHandler('debug.log', 'w+')
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.WARNING)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

In [ ]:
import neo4Polymer

In [ ]:
# simulation project function
def insertCodmucSimRunInGraph(myNeo4Polymer, simProject, simRunName, filepath, generation, spacerLength, graftedChainLength):
    '''insert all information of a codendrimer simulation Run (Codmuc) to the database'''
    myNeo4Polymer.addSimulationRunToSimulationProject(currentSimRun,currentProject)
    myNeo4Polymer.addBFMFileToDatabase(currentSimRun,filepath)
    #myNeo4Polymer.addDendrimerGenerationToSimulationRun(currentSimRun,generation)
    #myNeo4Polymer.addDendrimerSpacerLengthToSimulationRun(currentSimRun,spacer)
    myNeo4Polymer.addGraftedChainLengthToSimulationRun(currentSimRun,graftedChainLength)

# this ALWAYS deletes the full graph! Use this function with care
graph.delete_all()

# setup instacne of neo4j - BFM file interface
myNeoPolymers = neo4Polymer.neo4BFMPolymer(graph)

In [ ]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Linear dendritic copolymers
### -- preparation of project -- ###
# add simulation project:
currentProject = "Linear-Dendritic Copolymer Solutions"
myNeoPolymers.addNewSimulationProject(currentProject)
# add grafted chain as polymer:
myNeoPolymers.addNewPolymer("Grafted Chain")
# add dendrimer as polymer:
myNeoPolymers.addNewPolymer("Dendrimer")
# connect Dendrimers:
myNeoPolymers.connectSimulationToPolymer("Dendrimer", currentProject)
# connect Grafted Chain:
myNeoPolymers.connectSimulationToPolymer("Grafted Chain", currentProject)


### -- read in all information of simRun 1 -- ###
generation = 3
spacer = 4
numMolecules = 27
graftedChainLength = 4
currentSimRun = "codmuc_micelle_g{}_s{}_m{}_l{}_e0.8_b128".format(generation,spacer,numMolecules,graftedChainLength)
filepath = "bfm_files/{}_1425000000_lastconfig.bfm".format(currentSimRun)
# use simulation project function:
insertCodmucSimRunInGraph(myNeoPolymers, currentProject, currentSimRun, filepath, generation, spacer, graftedChainLength)
# add radius of gyration results
filepath = "bfm_files/rg_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addAnyRadiusOfGyrationFileToDatabase(currentSimRun,filepath)
#myNeoPolymers.addCODMUCRgTensorFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 2 -- ###
generation = 4
spacer = 4
numMolecules = 12
graftedChainLength = 4
currentSimRun = "codmuc_micelle_g{}_s{}_m{}_l{}_e0.8_b128".format(generation,spacer,numMolecules,graftedChainLength)
filepath = "bfm_files/{}_2225000000_lastconfig.bfm".format(currentSimRun)
# use simulation project function:
insertCodmucSimRunInGraph(myNeoPolymers, currentProject, currentSimRun, filepath, generation, spacer, graftedChainLength)
# add radius of gyration results
filepath = "bfm_files/rg_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addAnyRadiusOfGyrationFileToDatabase(currentSimRun,filepath)
#myNeoPolymers.addCODMUCRgTensorFileToDatabase(currentSimRun,filepath)


In [ ]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Ring opening Polymerisation
### -- preparation of project -- ###
# add simulation project:
currentProject = "Ring-Opening Polymerisation"
myNeoPolymers.addNewSimulationProject(currentProject)
# add ring polymer as polymer:
myNeoPolymers.addNewPolymer("Ring Polymer")
# connect Ring Polymer:
myNeoPolymers.connectSimulationToPolymer("Ring Polymer", currentProject)

### -- read in all information of simRun 1 -- ###
#RingMelt_N128_Phi05_Box128_EA5_EB17_MCS0_GPU
ring_size = 128
volume_fraction = "05"
activation_energy = 5
breaking_energy = 17

currentSimRun = "RingMelt_N{}_Phi{}_Box128_EA{}_EB{}_MCS0_GPU".format(ring_size,volume_fraction,activation_energy,breaking_energy)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 2 -- ###
#RingMelt_N128_Phi05_Box128_EA6_EB19_MCS0_GPU.bfm
ring_size = 128
volume_fraction = "05"
activation_energy = 6
breaking_energy = 19

currentSimRun = "RingMelt_N{}_Phi{}_Box128_EA{}_EB{}_MCS0_GPU".format(ring_size,volume_fraction,activation_energy,breaking_energy)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

In [ ]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Single Dendrimer Variable solvent Quality
### -- preparation of project -- ###
# add simulation project:
currentProject = "Single Dendrimers Solvent Quality"
myNeoPolymers.addNewSimulationProject(currentProject)
# add dendrimer as polymer:
myNeoPolymers.addNewPolymer("Dendrimer")
# connect Dendrimer:
myNeoPolymers.connectSimulationToPolymer("Dendrimer", currentProject)

### -- read in all information of simRun 1 -- ###
#dendr_g4_s4_b64_solv_e05.bfm
generation = 4
spacer = 4
nn_ineraction = "05" # = 0.5 k_B T

currentSimRun = "dendr_g{}_s{}_b64_solv_e{}".format(generation,spacer,nn_ineraction)
filepath = "bfm_files/bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)
# add radius of gyration file
filepath = "bfm_files/rg_files/{}_rg_mol.dat".format(currentSimRun)
myNeoPolymers.addAnyRadiusOfGyrationFileToDatabase(currentSimRun, filepath)
# add bond length file
filepath = "bfm_files/bondLength_files/{}_bl.dat".format(currentSimRun)
myNeoPolymers.addAnyBondLengthFileToDatabase(currentSimRun, filepath)

### -- read in all information of simRun 1 -- ###
#dendr_g4_s4_b64_solv_e08.bfm
generation = 4
spacer = 4
nn_ineraction = "80" # = 0.8 k_B T

currentSimRun = "dendr_g{}_s{}_b64_solv_e{}".format(generation,spacer,nn_ineraction)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)
# add radius of gyration file
filepath = "bfm_files/rg_files/{}_rg_mol.dat".format(currentSimRun)
myNeoPolymers.addAnyRadiusOfGyrationFileToDatabase(currentSimRun, filepath)
# add bond length file
filepath = "bfm_files/bondLength_files/{}_bl.dat".format(currentSimRun)
myNeoPolymers.addAnyBondLengthFileToDatabase(currentSimRun, filepath)

In [ ]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Single Dendrimer in Polymer Solution Umbrella Sampling
### -- preparation of project -- ###
# add simulation project:
currentProject = "Single Dendrimer-Linear Chain Solution Umbrella"
myNeoPolymers.addNewSimulationProject(currentProject)
# add Linear Chain as polymer:
myNeoPolymers.addNewPolymer("Linear Chain")
# connect Polymers:
myNeoPolymers.connectSimulationToPolymer("Dendrimer", currentProject)
myNeoPolymers.connectSimulationToPolymer("Linear Chain", currentProject)

### -- read in all information of simRun 1 -- ###
# dend_chain_spring_g3_s4_l32_b64_d05_r08_k1.0.bfm
generation = 3
spacer = 4
linear_chain_length = 32
spring_lenght = "08"
spring_const = "1.0"
vol_frac = "05" # = 0.5 k_B T

currentSimRun = "dend_chain_spring_g{}_s{}_l{}_b64_d{}_r{}_k{}".format(generation,spacer,linear_chain_length,vol_frac,spring_lenght,spring_const)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 2 -- ###
# dend_chain_spring_g3_s4_l32_b64_d05_r012_k1.0.bfm
generation = 3
spacer = 4
linear_chain_length = 32
spring_lenght = "012"
spring_const = "1.0"
vol_frac = "05" # = 0.5 k_B T

currentSimRun = "dend_chain_spring_g{}_s{}_l{}_b64_d{}_r{}_k{}".format(generation,spacer,linear_chain_length,vol_frac,spring_lenght,spring_const)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 3 -- ###
# dend_chain_spring_g3_s4_l32_b64_d05_r020_k1.0.bfm
generation = 3
spacer = 4
linear_chain_length = 32
spring_lenght = "020"
spring_const = "1.0"
vol_frac = "05" # = 0.5 k_B T

currentSimRun = "dend_chain_spring_g{}_s{}_l{}_b64_d{}_r{}_k{}".format(generation,spacer,linear_chain_length,vol_frac,spring_lenght,spring_const)
filepath = "bfm_files/{}.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)

In [ ]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Linear chains
### -- preparation of project -- ###
# add simulation project:
currentProject = "Linear Chain Solution"
myNeoPolymers.addNewSimulationProject(currentProject)
# add Linear Chain as polymer:
myNeoPolymers.addNewPolymer("Linear Chain")
# connect Linear chain:
myNeoPolymers.connectSimulationToPolymer("Linear Chain", currentProject)

### -- read in all information of simRun 1 -- ###
#linchains_n32_b64_d0.24_lastconfig.bfm
chain_length = 32
box_size = 64
volume_fraction = "0.24"

currentSimRun = "linchains_n{}_b{}_d{}".format(chain_length,box_size,volume_fraction)
filepath = "bfm_files/{}_lastconfig.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)
# add additional info
myNeoPolymers.addLinearChainLengthToSimulationRun(currentSimRun,chain_length)
myNeoPolymers.addVolumeFractionToSimulationRun(currentSimRun,volume_fraction)
# add radius of gyration
filepath = "bfm_files/rg_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addAnyRadiusOfGyrationFileToDatabase(currentSimRun,filepath)
# add bond length file
filepath = "bfm_files/bondLength_files/{}_bl.dat".format(currentSimRun)
myNeoPolymers.addAnyBondLengthFileToDatabase(currentSimRun, filepath)

### -- read in all information of simRun 2 -- ###
#linchains_n32_b64_d0.36_lastconfig.bfm
chain_length = 32
box_size = 64
volume_fraction = "0.36"

currentSimRun = "linchains_n{}_b{}_d{}".format(chain_length,box_size,volume_fraction)
filepath = "bfm_files/{}_lastconfig.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)
# add additional info
myNeoPolymers.addLinearChainLengthToSimulationRun(currentSimRun,chain_length)
myNeoPolymers.addVolumeFractionToSimulationRun(currentSimRun,volume_fraction)
# add radius of gyration
filepath = "bfm_files/rg_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addAnyRadiusOfGyrationFileToDatabase(currentSimRun,filepath)
# add bond length file
filepath = "bfm_files/bondLength_files/{}_bl.dat".format(currentSimRun)
myNeoPolymers.addAnyBondLengthFileToDatabase(currentSimRun, filepath)

In [ ]:
# -------------------------------------------------------------------------------------------------- #
# -------------------------------------------------------------------------------------------------- #
# Single Bottlebrushes
### -- preparation of project -- ###
# add simulation project:
currentProject = "Single Bottlebrushes"
myNeoPolymers.addNewSimulationProject(currentProject)
# add Bottlebrush as polymer:
myNeoPolymers.addNewPolymer("Bottlebrush")
# connect Bottlebrush:
myNeoPolymers.connectSimulationToPolymer("Bottlebrush", currentProject)

### -- read in all information of simRun 1 -- ###
# bottle_bb109_sc32_gd2_b256_lastconfig.bfm
Nbackbone = 109
Nsidechain = 32
graftigDensity = 0.5 # means one side chain at each second (1/2) backbone monomer
invGraftingDensity = 2
boxsize = 256

currentSimRun = "bottle_bb{}_sc{}_gd{}_b{}".format(Nbackbone,Nsidechain,invGraftingDensity,boxsize)
filepath = "bfm_files/{}_lastconfig.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)
# add additional info
myNeoPolymers.addNumberOfBackboneMonomersToSimulationRun(currentSimRun,Nbackbone)
myNeoPolymers.addNumberOfSideChainMonomersToSimulationRun(currentSimRun,Nsidechain)
myNeoPolymers.addGraftingDensityToSimulationRun(currentSimRun,graftigDensity)
# add bond length file
filepath = "bfm_files/bondLength_files/{}_bl.dat".format(currentSimRun)
myNeoPolymers.addAnyBondLengthFileToDatabase(currentSimRun, filepath)
# add radius of gyration
filepath = "bfm_files/rg_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addAnyRadiusOfGyrationFileToDatabase(currentSimRun,filepath)

### -- read in all information of simRun 2 -- ###
# bottle_bb73_sc32_gd2_b256_lastconfig.bfm
Nbackbone = 73
Nsidechain = 32
graftigDensity = 0.5 # means one side chain at each second (1/2) backbone monomer
invGraftingDensity = 2
boxsize = 256

currentSimRun = "bottle_bb{}_sc{}_gd{}_b{}".format(Nbackbone,Nsidechain,invGraftingDensity,boxsize)
filepath = "bfm_files/{}_lastconfig.bfm".format(currentSimRun)
# add simulation run node
myNeoPolymers.addSimulationRunToSimulationProject(currentSimRun,currentProject)
# read in bfm file
myNeoPolymers.addBFMFileToDatabase(currentSimRun,filepath)
# add additional info
myNeoPolymers.addNumberOfBackboneMonomersToSimulationRun(currentSimRun,Nbackbone)
myNeoPolymers.addNumberOfSideChainMonomersToSimulationRun(currentSimRun,Nsidechain)
myNeoPolymers.addGraftingDensityToSimulationRun(currentSimRun,graftigDensity)
# add bond length file
filepath = "bfm_files/bondLength_files/{}_bl.dat".format(currentSimRun)
myNeoPolymers.addAnyBondLengthFileToDatabase(currentSimRun, filepath)
# add radius of gyration
filepath = "bfm_files/rg_files/{}_rg.dat".format(currentSimRun)
myNeoPolymers.addAnyRadiusOfGyrationFileToDatabase(currentSimRun,filepath)

## Evaluation tools:
The class neo4PolymerRequests provides some helpful functions applying frequently used queries to an existing database and return the results as DataFrame.

In [ ]:
askme = neo4Polymer.neo4PolymerRequests(graph)

In [ ]:
# find simulation Projects for a certain polymer
polymer = "UnknownPolymer"
askme.getSimulationProjectsByPolymer(polymer)
polymer = "Dendrimer"
result_dataFrame = askme.getSimulationProjectsByPolymer(polymer)
print(result_dataFrame)

In [ ]:
# find all simulation Runs of a certain simulation Project
simProject = "Nonexisting Project"
askme.getSimulationRunsBySimulationProject(simProject)
simProject = "Single Dendrimer-Linear Chain Solution Umbrella"
result_dataFrame = askme.getSimulationRunsBySimulationProject(simProject)
print(result_dataFrame)